## EJERCICIO 9 - PRACTICA 3

### Importo librerias

In [40]:
ColabNotebook = 'google.colab' in str(get_ipython())

if ColabNotebook:
    # monta G-drive en entorno COLAB
    from google.colab import drive
    drive.mount('/content/drive/')

    # carpeta donde se encuentran archivos .py auxiliares
    FUENTES_DIR = '/content/drive/MyDrive/Colab Notebooks/FUENTES/'
    DATOS_DIR = '/content/drive/MyDrive/Colab Notebooks/DATOS/'      # carpeta donde se encuentran los datasets
else:
    # configuración para notebook con instalación LOCAL
    FUENTES_DIR = '../Fuentes'         # carpeta donde se encuentran archivos .py auxiliares
    DATOS_DIR   = '../Datos/' # carpeta donde se encuentran los datasets

# agrega ruta de busqueda donde tenemos archivos .py
import sys
sys.path.append(FUENTES_DIR)

### Leo dataframe

In [43]:
import pandas as pd      # para trabajar con archivos de datos csv, excel, etc: https://pandas.pydata.org/docs/getting_started/tutorials.html
import chardet           # para detectar la codificación de texto en archivos
import numpy as np

from sklearn import preprocessing
from matplotlib import pyplot as plt
import seaborn as sns

# importa neurona lineal de Laura en la carpeta Colab definida con FUENTES_DIR
from ClassNeuronaLineal import NeuronaLineal

nombre_archivo = DATOS_DIR + 'SEMILLAS.csv' # archivo de hojas

#-- detectando la codificación de caracteres usada ----
with open(nombre_archivo, 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large

# recupera el archivo en un objeto dataframe de pandas utilizando la codificación detectada
df = pd.read_csv(nombre_archivo, encoding=result['encoding'])

df.head()

,Area,Perimetro,Compacidad,LongNucleo,AnchoNucleo,Asimetria,LongSurco,Clase
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,Tipo1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,Tipo1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,Tipo1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,Tipo1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,Tipo1


In [45]:
# El atributo "Clase" contiene la clasificacion de la semilla, veamos cuantos tipos de clases existes
df['Clase'].unique() 

array(['Tipo1', 'Tipo2', 'Tipo3'], dtype=object)

#### Se puede ver que existen tres tipos de clases, pero se quiere predecir unicamente si una semilla es de clase "Tipo 2", por lo que se debe generar una columna binaria, que indique si la semilla es de Tipo 2

In [48]:
df_one_hot = pd.get_dummies(df['Clase'], prefix='Clase').astype(int) #genera una nueva columna binaria por cada tipo de clase
df_one_hot.head()

,Clase_Tipo1,Clase_Tipo2,Clase_Tipo3
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [50]:
#agrego las columnas binarias generadas al datframe, y elimino la columna "Clase"
df = pd.concat([df,df_one_hot], axis=1)
df = df.drop('Clase', axis=1)
df.head()

,Area,Perimetro,Compacidad,LongNucleo,AnchoNucleo,Asimetria,LongSurco,Clase_Tipo1,Clase_Tipo2,Clase_Tipo3
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1,0,0
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1,0,0
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1,0,0
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1,0,0
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1,0,0


### Genero los arreglos X e Y (entradas y valores a predecir)

In [63]:
Y = df['Clase_Tipo2']
X = df.drop(columns=['Clase_Tipo2']) # datos de entrada
X = np.array(X)
Y = np.array(Y)

### Normalizacion de datos de entrada

In [66]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

### Entrenamiento (uso el 60% de los datos para entrenamiento)

In [69]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.60, random_state=42)

In [71]:
X_train

array([[-7.91495401e-01, -7.59326908e-01, -6.44742463e-01,
        -6.68616399e-01, -7.71215300e-01,  4.79208003e-01,
        -4.73316767e-01, -7.07106781e-01,  1.41421356e+00],
       [-7.12260557e-01, -7.59326908e-01,  2.55133544e-02,
        -8.29247228e-01, -6.17293227e-01, -1.68283097e+00,
        -1.27689180e+00,  1.41421356e+00, -7.07106781e-01],
       [ 9.04228704e-02, -1.48027628e-02,  1.17088722e+00,
        -1.12064513e-01,  5.39776150e-01, -3.81474178e-01,
        -4.65158645e-01,  1.41421356e+00, -7.07106781e-01],
       [ 2.48892560e-01,  4.53403143e-01, -7.76248352e-01,
         6.59415948e-01, -7.32583134e-02, -7.06813376e-01,
         9.60473205e-01, -7.07106781e-01, -7.07106781e-01],
       [-4.50441070e-01, -5.44412722e-01,  6.83042796e-01,
        -6.34680309e-01, -2.72295477e-01, -7.42147346e-01,
        -4.69237706e-01,  1.41421356e+00, -7.07106781e-01],
       [ 1.23416063e+00,  1.08279469e+00,  1.55692063e+00,
         7.95160311e-01,  1.35980926e+00, -4.774759

In [75]:
from ClassNeuronaGral import NeuronaGradiente

### Utilizando funcio sigmoid() como activacion

In [89]:
train_accuracies = []
test_accuracies = []

iteraciones_promedio = []
iteracion_mejor_accuracy_train = []
veces_100_train = 0
veces_100_test = 0


for i in range(30):
    #division de datos
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.60, random_state=42)
    #modelo
    modelo_sigmoid = NeuronaGradiente(alpha=0.3, n_iter=200, FUN='sigmoid', COSTO='EC_binaria')
    #entrenamiento
    modelo_sigmoid.fit(X_train, y_train)
    #predicciones
    y_train_pred = modelo_sigmoid.predict(X_train)
    y_test_pred = modelo_sigmoid.predict(X_test)
    #accuracy
    train_accuracy = modelo_sigmoid.accuracy(X_train, y_train)
    test_accuracy = modelo_sigmoid.accuracy(X_test,y_test)
    #guardo los resultados
    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)

    # Encontrar la iteración con mejor accuracy
    best_train_iter = np.argmax(modelo_sigmoid.accuracy_)  # Encontrar la iteración donde se alcanzó el mejor accuracy en entrenamiento
    iteracion_mejor_accuracy_train.append(best_train_iter)
    # Contar si se alcanzó 100% de accuracy
    if train_accuracy == 1.0:
        veces_100_train += 1
    if test_accuracy == 1.0:
        veces_100_test += 1

    iteraciones_promedio.append(len(modelo_sigmoid.errors_))


#calculo el promedio de los accuracys
train_mean = np.mean(train_accuracies)
train_std = np.std(train_accuracies)
test_mean = np.mean(test_accuracies)
test_std = np.std(test_accuracies)
iteraciones_mean = np.mean(iteraciones_promedio)
iter_mejor_accuracy_mean = np.mean(iteracion_mejor_accuracy_train)


print(f'Iteraciones promedio: {iteraciones_mean:.2f}')
print(f'Precisión promedio en entrenamiento: {train_mean:.4f} ± {train_std:.4f}')
print(f'Precisión promedio en prueba: {test_mean:.4f} ± {test_std:.4f}')
print(f'Iteración promedio con mejor accuracy en train: {iter_mejor_accuracy_mean:.2f}')
print(f'Veces con 100% de accuracy en train: {veces_100_train} de 30')
print(f'Veces con 100% de accuracy en test: {veces_100_test} de 30')
print(' ')

Iteraciones promedio: 200.00
Precisión promedio en entrenamiento: 1.0000 ± 0.0000
Precisión promedio en prueba: 1.0000 ± 0.0000
Iteración promedio con mejor accuracy en train: 1.10
Veces con 100% de accuracy en train: 30 de 30
Veces con 100% de accuracy en test: 30 de 30
 


### Utilizando funcio tanh() como activacion

In [91]:
train_accuracies = []
test_accuracies = []

iteraciones_promedio = []
iteracion_mejor_accuracy_train = []
veces_100_train = 0
veces_100_test = 0


for i in range(30):
    #division de datos
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.40, random_state=42)
    #modelo
    modelo_tanh = NeuronaGradiente(alpha=0.3, n_iter=200, FUN='tanh', COSTO='EC_binaria')
    #entrenamiento
    modelo_tanh.fit(X_train, y_train)
    #predicciones
    y_train_pred = modelo_tanh.predict(X_train)
    y_test_pred = modelo_tanh.predict(X_test)
    #accuracy
    train_accuracy = modelo_tanh.accuracy(X_train, y_train)
    test_accuracy = modelo_tanh.accuracy(X_test,y_test)
    #guardo los resultados
    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)

    # Encontrar la iteración con mejor accuracy
    best_train_iter = np.argmax(modelo_tanh.accuracy_)  # Encontrar la iteración donde se alcanzó el mejor accuracy en entrenamiento
    iteracion_mejor_accuracy_train.append(best_train_iter)
    # Contar si se alcanzó 100% de accuracy
    if train_accuracy == 1.0:
        veces_100_train += 1
    if test_accuracy == 1.0:
        veces_100_test += 1

    iteraciones_promedio.append(len(modelo_tanh.errors_))


#calculo el promedio de los accuracys
train_mean = np.mean(train_accuracies)
train_std = np.std(train_accuracies)
test_mean = np.mean(test_accuracies)
test_std = np.std(test_accuracies)
iteraciones_mean = np.mean(iteraciones_promedio)
iter_mejor_accuracy_mean = np.mean(iteracion_mejor_accuracy_train)


print(f'Iteraciones promedio: {iteraciones_mean:.2f}')
print(f'Precisión promedio en entrenamiento: {train_mean:.4f} ± {train_std:.4f}')
print(f'Precisión promedio en prueba: {test_mean:.4f} ± {test_std:.4f}')
print(f'Iteración promedio con mejor accuracy en train: {iter_mejor_accuracy_mean:.2f}')
print(f'Veces con 100% de accuracy en train: {veces_100_train} de 30')
print(f'Veces con 100% de accuracy en test: {veces_100_test} de 30')
rprint(' ')

Iteraciones promedio: 1.00
Precisión promedio en entrenamiento: 0.3254 ± 0.0000
Precisión promedio en prueba: 0.3452 ± 0.0000
Iteración promedio con mejor accuracy en train: 0.00
Veces con 100% de accuracy en train: 0 de 30
Veces con 100% de accuracy en test: 0 de 30


C:\Users\W10\Deep Learning\clase 3\../Fuentes\ClassNeuronaGral.py:105: RuntimeWarning: invalid value encountered in log
  return(-y*np.log(y_hat+EPS)-(1-y)*np.log(1-y_hat+EPS))


NameError: name 'rprint' is not defined

### Resultados obtenidos

In [97]:
# Crear un DataFrame con las columnas indicadas
data = pd.DataFrame({
    'Funcion': ['sigmoid', 'sigmoid', 'sigmoid', 'tanh', 'tanh', 'tanh'],
    'alfa': [0.3, 0.2, 0.1, 0.3, 0.2, 0.1],
    'Iteraciones': [200, 200 , 200 ,1, 1 , 1 ],  
    'Accuracy train': [1.0,1.0,1.0,0.3,0.3,0.3],  
    'Ite accuracy': ['-','-','-','-','-','-'],  
    'Veces (datos Train)': [30,30,30,0,0,0],  
    'Accuracy test': ['?','?','?','?','?','?'],  
    'Veces (Datos Test)': ['?','?','?','?','?','?']
})
data
# Crear DataFrame
#df = pd.DataFrame(data)

# Mostrar el DataFrame
#print(df)

,Funcion,alfa,Iteraciones,Accuracy train,Ite accuracy,Veces (datos Train),Accuracy test,Veces (Datos Test)
0,sigmoid,0.3,200,1.0,-,30,?,?
1,sigmoid,0.2,200,1.0,-,30,?,?
2,sigmoid,0.1,200,1.0,-,30,?,?
3,tanh,0.3,1,0.3,-,0,?,?
4,tanh,0.2,1,0.3,-,0,?,?
5,tanh,0.1,1,0.3,-,0,?,?
